cookie
    - cookie是储存在客户端的一组键值对
    - cookie 有服务器创建的
    - cookie应用的简单示例
        - 免密登录 （指定时长内）

在爬虫中处理cookie的两种方式
 - 手动处理
    - 将cookie封装到headers字典中，将字典作用到get或者post的方法的headers参数中
 - 自动处理
    - Session对象
    - Session对象的创建：requests.Session()
    - 对象的作用：
        - 可以和requests一样调用get/post进行请求的发送。在使用session进行请求发送的过程中，如果产生了cookie，则cookie责备自动转存到session对选对象中
    - 在爬虫的过程中使用session处理cookie时，session对象至少需要用到几次？
        - 两次，第一次：捕获和储存cookie到session对象中，第二次：用携带cookie进行请求的发送

In [12]:
import  requests
header={
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'
}
url='https://xueqiu.com/statuses/hot/listV2.json?since_id=-1&max_id=484031&size=15'
url_json=requests.get(url=url,headers=header).json()
url_json

{'error_description': '遇到错误，请刷新页面或者重新登录帐号后再试',
 'error_uri': '/statuses/hot/listV2.json',
 'error_data': None,
 'error_code': '400016'}

结论：以上没有捕获到正确的数据，进行请求发送的时候没有在请求头中携带cookie

In [13]:
#基于session自动处理cookie
sess=requests.Session()
sess.get(url='https://xueqiu.com/',headers=header)
sess.get(url=url,headers=header)
url_json=sess.get(url=url,headers=header).json()
print(url_json)

{'next_max_id': 484016, 'items': [{'id': 484030, 'category': 0, 'data': None, 'column': '', 'original_status': {'id': 249146428, 'user_id': 1107854878, 'source': '雪球', 'title': '国轩高科李缜：碳酸锂不降价，新能源汽车、储能都没有未来', 'created_at': 1682699412000, 'retweet_count': 4, 'reply_count': 17, 'fav_count': 11, 'truncated': False, 'commentId': 0, 'retweet_status_id': 0, 'symbol_id': None, 'description': '作者|胡描 编辑|罗丽娟<br/> 4月28日凌晨，国轩高科发布了2022年报和2023年一季报。<br/> 财报显示，2022年公司营收突破两百亿，为230.52亿元，同比增长122.59%；营业利润达1.99亿元，同比上升408.87%；实现净利润3.66亿元，同比上升376.03%。<br/> 在今年一季报，国轩高科营收达71.77亿元，同比增...', 'type': '3', 'source_link': None, 'edited_at': None, 'user': {'subscribeable': False, 'common_count': 0, 'remark': None, 'recommend_reason': None, 'recommend': None, 'st_color': '1', 'intro': None, 'follow_me': False, 'allow_all_stock': False, 'stock_status_count': None, 'domain': None, 'type': '1', 'friends_count': 129, 'profile': '/1107854878', 'donate_count': 0, 'location': None, 'description': '', 'id': 1107854878, 'url': 

## 代理操作
- 概念：代理服务器
- 代理服务器的作用
    - fiddler是一种典型的代理服务器的抓包工具
    - 代理服务器就是用来转发请求和响应的，可以让代理服务器转发请求从而更换请求的原始IP地址
- 代理和爬虫之前的关系：
    - 爬虫程序可能会在短时间内对指定的服务器发起高频的请求，服务器端会将高频请求的IP禁掉
- 代理的匿名度
    - 透明：对方服务器：知道你使用了代理服务器，也知道你真实的IP地址
    - 匿名：对方服务器：知道你使用了代理服务器，但不知道你真实的IP地址
    - 高匿：对方服务器：不知道你使用了代理服务器，不也知道你真实的IP地址
- 代理的类型：
    - http：只能代理http协议的请求
    - https：只能代理https协议的请求

In [16]:
##构建代理池
from lxml import etree
import requests
from bs4 import BeautifulSoup
import os
import re
import time
import random

In [17]:
url='http://www.xiladaili.com/https/'                          #目标网站
basePath = os.getcwd()                                         #当前程序所在的路径
userAgentList = [                                              #user-agent列表
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.62 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0)",
    "Mozilla/5.0 (Macintosh; U; PPC Mac OS X 10.5; en-US; rv:1.9.2.15) Gecko/20110303 Firefox/3.6.15",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.25 Safari/537.36 Core/1.70.3823.400 QQBrowser/10.7.4307.400",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4315.4 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/14.0.835.163 Safari/535.1",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:6.0) Gecko/20100101 Firefox/6.0",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50",
    "Opera/9.80 (Windows NT 6.1; U; zh-cn) Presto/2.9.168 Version/11.50",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 2.0.50727; SLCC2; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; InfoPath.3; .NET4.0C; Tablet PC 2.0; .NET4.0E)",
]
timeout = 10                                                    #连接超时时间
ipList = {}                                                     #这里我把这两个列表也直接全局变量了
okList = {}                                                     #这里我把这两个列表也直接全局变量了

In [18]:
def get_one_page(url):
    global timeout
#     time.sleep(5)                                             #睡眠1秒，避免频繁范围
    headers={                                                   #在请求头中加入user-agent参数，通过random库，
        'User-Agent':random.choice(userAgentList),              #随机选取上面user-agent列表中的一条user-agent
    }
    response=requests.get(                                      #发起请求
        url,                                                    #目标网站
        headers=headers,                                        #请求头
#         timeout=timeout,                                      #超时时间
#         verify=False,                                         #不检测ssl证书
    )
    if response.status_code==200:                               #响应状态码为200的情况，表示访问成功
        response.close()                                        #按网上的方法来的，避免连接太多
#         print(response.text)
        return response.text                                    #返回目标网站的html代码
    return None

In [19]:
def parse_one_page(html):                                       #解析html部分
    global ipList,okList
    soup=BeautifulSoup(html,'html5lib')                         #用beautifulsoup进行解析，这个地方稍微有点不同，lxml解析不了，需要跳过html5lib来解析
    table = soup.find_all(class_="fl-table")[0]                 #选取html代码中class为fl-table的第一个标签
#     print(table)
    trList = table.select("tr")                                 #再选取其中标签为tr的所有标签
#     print(trList)
    for i in range(len(trList)):                                #循环获取tr的标签列表
        if (i != 0):                                            #由于第一个tr是表头中的tr，所以我们要去掉第一个tr，所以i要不等于0
            tdList = trList[i].select("td")                     #选取当前tr标签中的td标签
#             print(tdList)
#             print(tdList[1].next_element)
#             print(tdList[1].next_element+":"+tdList[2].next_element)
#             ip = tdList[0].next_element+":"+tdList[1].next_element
            ip = tdList[0].next_element                         #通过分析html代码我们可以知道，第一个td是我们需要的ip和端口号
#             print(ip)
            aDict = {"https":ip}                                #将ip做成一个字典（其实这么做是错的，字典的key只能唯一，所以会覆盖之前的ip）
            ok = checkIpIsOK(aDict)                             #测试这个ip能不能访问之后真正想爬取的网站（请看我下一篇博客）
            ipList.update(aDict)                                #将aDict存进ipList
            okList.update({ip:ok})                              #如果测试ip可以成功访问的，就把ip存进okList
    return ipList,okList                                        #这个return其实没用，因为这两个list已经是全局变量了，外面也可以看得到


In [20]:
def checkIpIsOK(ipList):                                    #检测ip是否可以成功访问的函数
    headers={
        'User-Agent':random.choice(userAgentList),
    }
    try:
        print(ipList)
        response=requests.get(
            url='https://unpkg.com/browse/element-ui@2.14.1/',
            headers=headers,
            timeout=timeout,
            verify=False,
            proxies=ipList,
        )
        if response.status_code==200:                        #只要状态码是200就说明可以正常访问，不需要做更多操作
            print("ok")
            response.close()
            return True
        return False
    except BaseException:
        return False


In [21]:
def main():
    global url
    for i in range(1):                                       #不需要循环，但是懒得改代码了，所以就循环改成1，相当于没有循环
        html=get_one_page(url)
        parse_one_page(html)
#     print(okList)
#     checkIpIsOK(ipList)

main()

TypeError: object of type 'NoneType' has no len()

# 问题
 在进行大量请求发送的时候，经常会出现得报错：HTTPConnection（host：xxx）Max retrieve execeed with url
 - 原因：
    - 每次数据传输前客户端和服务器建立TCP连接，为了节省传输消耗，默认为keep-alive，即：连接一次，传输多次，然后连接迟迟不断开的话，则连接池满后就不无法创建新的连接对象，到时请求无法发送
    - ip被封
    - 请求太频繁
 - 解决办法：
    - 设置请求头中的Connection的值为close，表示每次请求成功后断开连接
    - 更换请求ip
    - 没请求之前使用sleep进行等待间隔

In [ ]:
import requests
header={
    'connection':'close',
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'
}

# 验证码识别


# 模拟登陆

# 异步爬虫